# Principal Component Analysis

Principal Component Analysis is a technique for extracting information about variability in a dataset. It can be used on high-dimensional data as a form of *dimension reduction*.

In [ ]:
# Import some useful packages
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import seaborn as sns; sns.set()

We begin by creating some toy data.

In [ ]:
N = 500 # Number of points to sample
m = 0.5 # Slope
b = 2   # Intercept
x_shift = 4 # Horizontal shift

# Sample x values according to a normal distribution, shift them horizonatally and sort them
xs = np.random.normal(0,1,N) + x_shift
xs = np.sort(xs)

# Get y values by applying a linear map to the x values then adding noise
ys = m*xs + b + np.random.normal(0,0.5,N)
plt.axis('equal')

plt.scatter(xs,ys)
plt.axis('equal');

Now we have a *point cloud*; this is common terminology for a collection of vectors in some vector space.  Intuitively, there is one direction which explains most of the variation in the data (a vector in the direction of the best fit line). The rest of the variation is in the "thickness" of the point cloud, which we can visualize as a shorter vector orthogonal to the first one. The goal of *principal component analysis (PCA)* is to determine these vectors.

## PCA: Basic Theory

Let's make our goal precise. We will organize our data as a "tall, skinny matrix" $X \in \mathbb{R}^{N \times d}$, with each of the $N$ ($N=500$ in our example) rows giving a vector in $\mathbb{R}^d$ ($d = 2$ in our example). This is called the *data matrix*. Denote the rows by $\vec{x}_1,\ldots,\vec{x}_N$, each $\vec{x}_j \in \mathbb{R}^d$.

In [ ]:
X = np.array([xs,ys]).T # Form a "tall, skinny matrix" containing the data
X.shape

It is standard to assume that the columns of $X$ have mean zero. This loses no generality, because we can preprocess by shifting, then shift back at the end if we want to.

In [ ]:
mu0 = np.mean(X[:,0])
mu1 = np.mean(X[:,1])

X_centered = X - np.array([mu0,mu1])

In [ ]:
plt.scatter(X[:,0],X[:,1])
plt.scatter(X_centered[:,0],X_centered[:,1])
plt.axis('equal');

The first goal is to find a unit vector which lines up as closely with as many of the $\vec{x}_j$'s as it possibly can. That is, we want to find 
$$
\vec{v}_1 = \mathrm{argmax}_{\|\vec{v}\| = 1} \sum_j (\vec{x}_j \cdot \vec{v})^2.
$$
This defines our first *principal vector*. 

To get the next principal vector, we look for the direction of greatest variation which is orthogonal to $\vec{v}_1$. More precisely, the second principal vector $\vec{v}_2$ is 
$$
\vec{v}_2 = \mathrm{argmax} \left\{ \sum_j (\vec{x}_j \cdot \vec{v})^2 : \|\vec{v}\| = 1 \mbox{ and } \vec{v} \cdot \vec{v}_1 = 0\right\}.
$$
This can be continued inductively to find all principal vectors $\vec{v}_1,\vec{v}_2, \ldots, \vec{v}_d$. 

Digging into the math a bit more, note that
\begin{align*}
\max_{\|\vec{v}\| = 1} \sum_j (\vec{x}_j \cdot \vec{v})^2 &= \max_{\|\vec{v}\| = 1} \left\|X \vec{v} \right\|^2 \\
&= \max_{\|\vec{v}\| = 1} \vec{v}^T X^T X \vec{v} \\
&= \max_{\vec{v} \neq \vec{0}} \frac{\vec{v}^T X^T X \vec{v}}{\|\vec{v}\|^2}.
\end{align*}
It is not hard to show that this max value is the maximum eigenvalue of $X^T X$, realized by the corresponding (unit)  eigenvector. Similarly, the remaining singular vectors are obtained as the other eigenvectors of $X^T X$ (arranged in decreasing order of eigenvalue).



The matrix $X^T X \in \mathbb{R}^{d \times d}$ is called the *covariance matrix of $X$*.

#### Conclusion

The singular vectors of the centered data matrix $X$ are the unit eigenvectors (well-defined up to sign $\pm 1$) of the covariance matrix $X^T X$, listed in descending order of corresponding eigenvalue.

In [ ]:
cov = X_centered.T@X_centered
cov

We can easily compute the eigenvalues of the covariance matrix using `numpy`, as we do below. The output of `np.linalg.eig` is a "tuple" `eVals, eVec`. The variable `eVals` stores the eigenvalues in descending order. The variable `eVec` stores the corresponding eigenvectors as **columns**. The eigenvectors are normalized (i.e., they are unit vectors).

In [ ]:
eVals, eVec = np.linalg.eig(cov)
print(eVals)
print(eVec)

Note the the "slope" of the first eigenvector is pretty close to the slope that used to construct our data.

In [ ]:
eVec[1,0]/eVec[0,0]

We can plot the principal vectors over the pointcloud using the `quiver` function. The principal vectors are scaled by their respective eigenvalues to illustrate the difference in variability quantification. 

The `scale` option has been tuned to give a good picture (higher values scale the vectors down more --- note that the norms of the true principal vectors are quite large!). The same scale is used for both vectors to illustrate their relative lengths.

In [ ]:
plt.scatter(X_centered[:, 0], X_centered[:, 1], alpha=0.2)
# alpha controls 'opacity' of the points

# Syntax for quiver:
# plt.quiver(xVal for basepoint, yVal for basepoint, xVal for vector, yVal for vector, scale = )
plt.quiver(0, 0, eVals[0]*eVec[0,0], eVals[0]*eVec[1,0], scale=1200)
plt.quiver(0, 0, eVals[1]*eVec[0,1], eVals[1]*eVec[1,1], scale=1200)

plt.axis('equal');

We could also shift everything back to the original data.

In [ ]:
plt.scatter(X[:, 0], X[:, 1], alpha=0.2)

# Syntax for quiver:
# plt.quiver(xVal for basepoint, yVal for basepoint, xVal for vector, yVal for vector, scale = )
plt.quiver(mu0, mu1, eVals[0]*eVec[0,0], eVals[0]*eVec[1,0], scale=1200)
plt.quiver(mu0, mu1, eVals[1]*eVec[0,1], eVals[1]*eVec[1,1], scale=1200)

plt.axis('equal');

## PCA with SVD

The *Singular Value Decomposition (SVD)* expresses $X$ as 
$$
X = U \Sigma W^T,
$$
where $U \in \mathbb{R}^{N \times N}$ and $W \in \mathbb{R}^{d \times d}$ are matrices with orthonormal column vectors (orthogonal matrices) and $\Sigma \in \mathbb{R}^{N \times d}$ is a diagonal matrix with the square roots of the eigenvalues of $X^T X$ on its diagonal. These are called the *singular values* of $X$.

We calculate the SVD of our centered data using the code below.

In [ ]:
U, Sigma, Wt = np.linalg.svd(X_centered)

In [ ]:
print(U.shape)
print(Sigma.shape)
print(Wt.shape)

Looking at the shapes, we see that `Sigma` stores only the nonzero entries of $\Sigma$ (not the full matrix in $\mathbb{R}^{N \times d}$). These are given in descending order.

In [ ]:
Sigma

If we square these values, we should get the eigenvalues of $X^T X$ that we computed earlier.

In [ ]:
print(Sigma**2) # Note that this syntax applies the **2 operation to each entry in the array
print(eVals)

And if we examine the matrix $W^T$, we see that its columns are (up to a sign) the same as the eigenvectors of $X^T X$ we computed earlier!

In [ ]:
print(Wt)
print(eVec)

Indeed, this must be the case in general, since
\begin{align*}
X^T X &= (U\Sigma W^T)^T (U \Sigma W^T) \\
&= W \Sigma^T U^T U \Sigma W^T \\
&= W \Sigma^T \Sigma W^T \\
&= W \widehat{\Sigma} W^T,
\end{align*}
where $\widehat{\Sigma}$ is a diagonal matrix containing the eigenvalues of $X^T X$. It follows that the columns of $W^T$ are the eigenvectors of $X^T X$. 

#### Conclusion

The principal vectors of $X$ are given by the columns of $W^T$ from the SVD of $X$ (read from left-to-right).

### Theoretical Homework

Verify the steps and statements made in the above proof.

### Exercise

Write a function to compute the principal vectors of a 2-dimensional point cloud. Use it to perform PCA on the examples `X1`, `X2` and `X3` defined below. Plot the results.

For plotting purposes, it may be useful to have your function return a 'tuple'; e.g.
`return X_centered, Sigma, Wt`
or something along those lines.

In [ ]:
# Example 1 generation
mean1 = [0, 0]
cov1 = [[10, 0], [0, 10]]

X1 = np.random.multivariate_normal(mean1, cov1, 500)

# Example 2 generation
mean2 = [2, -3]
cov2 = [[1, 1], [1, 10]]

X2 = np.random.multivariate_normal(mean2, cov2, 500)

# Example 3 generation
xs = np.random.uniform(0,2*np.pi,500)
xs = np.sort(xs)
ys = 2*np.sin(xs) + np.random.normal(0,0.5,500)
X3 = np.array([xs,ys]).T

fig = plt.figure(figsize=(10,5))

fig.add_subplot(1,3,1)
plt.scatter(X1[:,0],X1[:,1])
plt.axis('equal')

fig.add_subplot(1,3,2)
plt.scatter(X2[:,0],X2[:,1])
plt.axis('equal')

fig.add_subplot(1,3,3)
plt.scatter(X3[:,0],X3[:,1])
plt.axis('equal')

plt.show()

## PCA with SciKit-Learn

Now that we have a solid theoretical understanding of PCA, we can use a built in function from `scikit-learn` to do the computation. This will work in arbitrary dimension, unlike the function we created above.

Let's try it on the dataset `X` we have been using to make sure it agrees.

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2) # Specify the number of principal directions you want to find
pca.fit(X)

Check the principal vectors with `pca.components_` and compare to our earlier result.

In [ ]:
print(pca.components_)
print(eVec)

Careful! We should notice that the principal vectors we are after are given by the *rows* of `pca.components_`. Previously we had been using *columns* of matrices. Once we notice that difference, everything agrees up to a sign.

The `pca.singular_values_` method pulls the singular values of $X$, or the square roots of the eigenvalues of $X^T X$.

In [ ]:
pca.singular_values_**2

Perhaps a more useful way to compare how variance is captured by each principal vector is to look ath the *explained variance ratio*. If $\lambda_1,\ldots,\lambda_d$ are the eigenvalues of $X^T X$ (listed in descending order), then the explained variance ratios are given by $\frac{\lambda_j}{\sum_k \lambda_k}$.

In [ ]:
pca.explained_variance_ratio_

## PCA as Dimension Reduction

Let's return to our favorite dataset, MNIST.

In [ ]:
from sklearn.datasets import load_digits
digits = load_digits()
digits.data.shape

We can think of the MNIST dataset as a point cloud in $\mathbb{R}^{8 \times 8} \approx \mathbb{R}^{64}$. This means it is impossible to visualize directly. On the other hand, we've seen that the data is structured enough that simple classifiation algorithms like logistic regression and SVM work extremely well. This leads us to believe that we may be able to get some sort of visualization of the data by exploiting its special structure.

We begin by applying PCA to MNIST.

In [ ]:
pca = PCA()
pca.fit(digits.data)

In [ ]:
pca.components_.shape

If we look at the explained variance ratios, we observe the following. Even though the data lives in $64$ dimensions, if we use the principal vector basis then any direction after the first 10 contributes less than 2\% of the total variance in the data!

In [ ]:
plt.plot(pca.explained_variance_ratio_)

If we only used the first couple of principal vectors as our coordinate axes, we might actually get a reasonable picture of the MNIST dataset!

In [ ]:
pVec0 = pca.components_[0]
pVec1 = pca.components_[1]

In [ ]:
pVec0

Even though each principal vector lives in $\mathbb{R}^64$, the first two of them still span a 2-dimensional plane. We can orthogonally project each point from the MNIST dataset onto this 2-dimensional plane. This is accomplished by taking dot products with each of the principal vectors.

In [ ]:
projectedMNIST = []

for j in range(1797):
    projectedMNIST.append([np.dot(digits.data[j],pVec0),np.dot(digits.data[j],pVec1)])

projectedMNIST = np.array(projectedMNIST)

Now we can plot the projected point cloud on this two dimensional plane.

In [ ]:
plt.figure(figsize=(10,10))

plt.scatter(projectedMNIST[:, 0], projectedMNIST[:, 1],
            c=digits.target, alpha=0.5,
            cmap=plt.cm.get_cmap('nipy_spectral_r', 10))

plt.xlabel('pVec0')
plt.ylabel('pVec1')
plt.axis('equal')
plt.colorbar();

This gives a relatively faithful picture of the 64-dimensional MNIST dataset in only 2-dimensions. We can see the separation in the classes and this makes it more clear why logistic regression and SVM did such a good job of separating the data.

The procedure of finding a low-dimensional representation of high-dimensional data is called *dimension reduction*.

Of course this dimension reduction procedure is built into `scikit-learn`. The same effect is achieved by the following code.

In [ ]:
pca = PCA(2)  # project from 64 to 2 dimensions
projected = pca.fit_transform(digits.data)

In [ ]:
from matplotlib.pyplot import figure

plt.figure(figsize = (10,10))

plt.scatter(projected[:, 0], projected[:, 1],
            c=digits.target, alpha=0.5,
            cmap=plt.cm.get_cmap('nipy_spectral_r', 10))

plt.xlabel('component 1')
plt.ylabel('component 2')
plt.axis('equal')
plt.colorbar();

### Exercise

Create a 3D scatterplot showing the projection of the MNIST data onto the first 3 principal directions. 

Hints:
- Look here for an idea of how to create 3D scatterplots https://matplotlib.org/2.1.1/gallery/mplot3d/scatter3d.html
- Use the command `%matplotlib notebook` to get a 3D plot which you can drag to rotate.

### Exercise

Apply similar dimension reduction techniques on the `fashion-mnist` and/or `olivetti_faces` datasets that we have studied previously. Are these datasets similarly well-separated?

### Exercise

Create "random digits" as follows. Create a data matrix containing only the vectors for a fixed choice of digit in MNIST. Center the data matrix then compute its covariance matrix. Use this as the covariance matrix in the multivariate normal sampling function (this was used above to create examples). A sampled point is your 'random digit'. You can visualize it by reshaping and using `plt.imshow`.

Try classifying your random digits with a classifier that you train on MNIST. How often are they correctly classified?

**Remark:** When I tried this my random digits looked pretty bad, although they were still recognizable. Can you find a way to improve this algorithm to produce nicer looking results?